# DS2002 - Nick Kellogg 

### Importing Libraries

In [1]:
import os
import json
import numpy
import datetime
import pandas as pd
import csv
import certifi

import pymongo
from sqlalchemy import create_engine

### Connecting to MongoDB and MySQL

In [9]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "airlines"
}

mongodb_args = {
    "user_name" : "vtf6hv",
    "password" : "6uC7IRMO3XXShk56",
    "cluster_name" : "dsproject",
    "cluster_subnet" : "k1xls5z",
    "cluster_location" : "atlas", # "local"
    "db_name" : "Kellogg-Database"
}

### Function declarations

In [10]:

def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(client, db_name, data_directory, csv_files):
    db = client[db_name]
        
    for collection_name, csv_file in csv_files.items():
        db[collection_name].drop()
        csv_path = os.path.join(data_directory, csv_file)
        try:
            df = pd.read_csv(csv_path, encoding='utf-8')  # try with utf-8
        except UnicodeDecodeError:
            df = pd.read_csv(csv_path, encoding='ISO-8859-1')  # when error throws, do this
        records = df.to_dict(orient='records')
        db[collection_name].insert_many(records)
    
    client.close()


### Populating Data

In [11]:
data_dir = os.path.join(os.getcwd(), 'dataset_airlines')

csv_files = {
    "airline_accidents": 'airline_accidents.csv',
    "faa_incident_data": 'faa_incidents_data.csv',
    "ntsb_aviation_data": 'ntsb_aviation_data.csv',
    "world_aircraft_accident_summary": 'world_aircraft_accident_summary.csv'
}

client = get_mongo_client(**mongodb_args)

set_mongo_collections(client, mongodb_args["db_name"], data_dir, csv_files)

/var/folders/7k/1xs66xvn4d9_vwy619vxxrjc0000gn/T/ipykernel_88351/1414483541.py:65: DtypeWarning: Columns (0,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='utf-8')  # try with utf-8


AutoReconnect: ac-ur6drbo-shard-00-01.k1xls5z.mongodb.net:27017: connection closed (configured timeouts: connectTimeoutMS: 20000.0ms)


### Creating dataframes

In [20]:
# Airline Accidents DF
client = get_mongo_client(**mongodb_args)

query = {}
collection = "airline_accidents"
df_airline_accidents = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_airline_accidents.head(2)


# FAA Incident Data DF
query = {}
collection = "faa_incident_data"
df_faa_incident_data = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_faa_incident_data.head(2)


# NTSB Aviation Data DF
query = {}
collection = "ntsb_aviation_data"
df_ntsb_aviation_data = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_ntsb_aviation_data.head(2)

# World Aircraft Accident Summary DF
query = {}
collection = "world_aircraft_accident_summary"
df_world_aircraft_accident_summary = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_world_aircraft_accident_summary.head(2)


InvalidOperation: Cannot use MongoClient after close

### Transformations to each Data Frame

In [18]:
# Df_Airline_accidents rename
df_airline_accidents.rename(columns = {"Event Id" : "accident_id",
                            "Event Date" : "event_date",
                            "Investigation Type" : "investigation_type",
                            "Injury Severity" : "injury_severity",
                            "Aircraft Damage" : "aircraft_damage",
                            "Purpose of Flight" : "flight_purpose",
                            "Total Major Injuries" : "total_fatal_injuries",
                            "Total Serious Injuries" : "total_serious_injuries",
                            "Total Minor Injuries" : "total_minor_injuries",
                            "Total Uninjured" : "total_uninjured",
                            "Weather Condition" : "weather_condition"
                            }, inplace = True)
df_airline_accidents.drop(['Accident Number', 'Latitude', 'Longitude', 'Airport Code', 'Airport Name', 'Aircraft Category', 'Make', 'Model', 'Amateur Built', 'Number of Engines', 'FAR Description', 'Schedule'], axis = "columns", inplace = True)
df_airline_accidents.head(2)

SyntaxError: expression expected after dictionary key and ':' (340310181.py, line 11)

In [ ]:
# FAA_Incidents rename
df_faa_incident_data.rename(columns = {'AIDS Report Number' : 'report_id',
                                       'Local Event Date' : 'faa_date',
                                       'Event City' : 'faa_event_city',
                                       'Event State' : 'faa_event state',
                                       'Event Airport': 'faa_airport'
                            }, inplace = True)
df_faa_incident_data.drop(['Event Type', 'Aircraft Damage','Flight Phase', 'Aircraft Make', 'Aircraft Series', 'Operator', 'Primary Flight Type', 'Flight Conduct Type', 'Flight Plan Filed Code', 'Aircraft Engine Make', 'Aircraft Engine Model', 'Engine Model', 'Nbr of Engines', 'PIC Certificate Type', 'PIC Flight Time Total Hrs', 'PIC Flight Time Total Make-Model'], axis = "columns", inplace = True)
df_faa_incident_data.head(2)

In [ ]:
# DF_World_Aircraft_accident_summary rename
df_world_aircraft_accident_summary.rename(columns = {'WAAS Subset Event Id' : 'waas_id',
                                       'Local Event Date' : 'waas_date',
                                       'Event Location' : 'waas_location',
                                       'Crew Fatalities' : 'waas_crew_fatalities',
                                       'Crew Injured' : 'waas_crew_injured',
                                       'PAX Fatalities' : 'waas_pax_fatalities',
                                       'PAX Injuries' : 'waas_pax_injuries',
                                       'PAX Aboard' : 'waas_pax_aboard'
                            }, inplace = True)
df_world_aircraft_accident_summary.drop(['Aircraft Operator', 'Aircraft'], axis = "columns", inplace = True)
df_world_aircraft_accident_summary.head(2)

In [ ]:
# df_ntsb_aviation_data rename 
df_ntsb_aviation_data.rename(columns = {'NTSB_RPRT_NBR' : 'ntsb_id',
                                       'EV_TYPE_DESC' : 'ntsb_event_type_desc',
                                       'EVENT_LCL_DATE' : 'ntsb_event_date',
                                       'LOC_STATE_CODE_STD' : 'ntsb_state_abbrev',
                                       'INJURY_DESC' : 'ntsb_injury_desc',
                            }, inplace = True)
df_ntsb_aviation_data.drop(['ACFT_REGIST_NBR', 'ACFT_REGIST_NBR', 'ACFT_SERIAL_NBR', 'APRT_NAME_STD', 'FLTCNDCT_DESC', 'OPRTR_SCHED_DESC', 'ACFT_NSDC_MAKE_STD', 'ACFT_NSDC_SERIES_STD', 'REPORT_STATUS', 'FLIGHT_PHASE_DESC'], axis = "columns", inplace = True)
df_ntsb_aviation_data.head(2)


### Putting Transformed Data into the Warehouse

In [ ]:
# Airline Accidents DF
dataframe = df_airline_accidents
table_name = 'dim_airline_accidents'
primary_key = 'accident_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

# FAA Incident Data DF
dataframe = df_faa_incident_data
table_name = 'dim_faa_incident_data'
primary_key = 'report_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

# World Aircraft Accident Summary DF
dataframe = df_world_aircraft_accident_summary
table_name = 'dim_world_aircraft_accident_summary'
primary_key = 'waas_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

# NTSB Aviation Data DF
dataframe = df_ntsb_aviation_data
table_name = 'dim_ntsb_aviation_data'
primary_key = 'ntsb_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

### Validating that they were created

In [ ]:
sql_airline_accidents = "SELECT * FROM dim_airline_accidents;"
df_dim_airline_accidents = get_sql_dataframe(sql_airline_accidents, **mysql_args)
df_dim_airline_accidents.head(2)

In [ ]:
sql_faa_incident_data = "SELECT * FROM dim_faa_incident_data;"
df_dim_faa_incident_data = get_sql_dataframe(sql_faa_incident_data, **mysql_args)
df_dim_faa_incident_data.head(2)

In [ ]:
sql_world_aircraft_accident_summary = "SELECT * FROM dim_world_aircraft_accident_summary;"
df_dim_world_aircraft_accident_summary = get_sql_dataframe(sql_world_aircraft_accident_summary, **mysql_args)
df_dim_world_aircraft_accident_summary.head(2)

In [ ]:
sql_ntsb_aviation_data = "SELECT * FROM dim_ntsb_aviation_data;"
df_dim_ntsb_aviation_data = get_sql_dataframe(sql_ntsb_aviation_data, **mysql_args)
df_dim_ntsb_aviation_data.head(2)

###  Setting up my SQL statement. The goal of this is to: 
1) Average Number of Fatalities for Each Investigation Type
2) Total Number of Accidents by Event Location
3) Total Fatalities and Injuries in a Specific Year (2020)

In [ ]:
sql_avg_fatalities_by_type = """
SELECT 
    investigation_type, 
    ROUND(AVG(total_fatal_injuries), 2) AS average_fatalities
FROM 
    dim_airline_accidents
GROUP BY 
    investigation_type;
"""

df_avg_fatalities_by_type = get_sql_dataframe(sql_avg_fatalities_by_type, **mysql_args)
df_avg_fatalities_by_type

In [ ]:
sql_total_accidents_by_location = """
SELECT 
    faa_event_city, 
    faa_event_state, 
    COUNT(*) AS total_accidents
FROM 
    dim_faa_incident_data
GROUP BY 
    faa_event_city, faa_event_state
ORDER BY 
    total_accidents DESC;
"""

df_total_accidents_by_location = get_sql_dataframe(sql_total_accidents_by_location, **mysql_args)
df_total_accidents_by_location

In [ ]:
sql_fatalities_injuries_2020 = """
SELECT 
    YEAR(event_date) AS year, 
    SUM(total_fatal_injuries) AS total_fatalities, 
    SUM(total_serious_injuries + total_minor_injuries) AS total_injuries
FROM 
    dim_airline_accidents
WHERE 
    YEAR(event_date) = 2020
GROUP BY 
    year;
"""

df_fatalities_injuries_2020 = get_sql_dataframe(sql_fatalities_injuries_2020, **mysql_args)
df_fatalities_injuries_2020

### Conclusion Statement
This whole project was done to essentially demonstrate my ability to combine datasets into one SQL database and then query it for different needs. The dataset I chose reflects airline accident reports from varying sources, however by combining all of them into one, it is easier to query and find out larger patterns within the data. The ETL pipeline (which won't connect anymore despite working earlier and also me trying 4 different ways!) is coded correctly and successfully manipulates the CSV data files to get the certain columns needed for the data that I wanted. 